In [1]:
import os
os.chdir("..")
%pwd

'c:\\Users\\Lenovo\\MLOPS\\HealthKart-recommendation-system'

In [2]:
import sys
import pandas as pd
import spacy
import re
from collections import Counter

In [3]:
import sys
print(sys.executable)


c:\Users\Lenovo\MLOPS\HealthKart-recommendation-system\.venv\Scripts\python.exe


In [4]:
!python -m ensurepip --upgrade
!python -m spacy download en_core_web_sm

Looking in links: c:\Users\Lenovo\AppData\Local\Temp\tmpsbpaqm8g
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
nlp = spacy.load("en_core_web_sm")

In [22]:
df = pd.read_csv("data/dataset.csv")
df.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [23]:
df = df.dropna(subset=['reviews.text'])
df = df.drop_duplicates()

In [28]:
df.shape

(71008, 25)

In [29]:
doc = nlp(str(df['reviews.text']))

In [30]:
for ent in doc.ents:
    if ent.label_ in ['ORG', 'PRODUCT']:
        print(ent.text)

2                                             Good


In [31]:
import os
import sys
import pandas as pd
import spacy
import re
from collections import Counter
from src.exception import CustomException
from src.logging import logging


class EntityExtraction:
    def __init__(
        self,
        data_path: str = "data/raw/dataset.csv",
        output_dir: str = "data/entity_data",
        output_file: str = "entity_extracted.csv"
    ):
        self.data_path = data_path
        self.output_dir = output_dir
        self.output_file = os.path.join(output_dir, output_file)
        
        # Load spaCy model
        try:
            self.nlp = spacy.load("en_core_web_sm")
            logging.info("SpaCy model loaded successfully")
        except:
            logging.info("Downloading SpaCy model...")
            os.system("python -m spacy download en_core_web_sm")
            self.nlp = spacy.load("en_core_web_sm")

    def extract_brands(self, row):
        """Extract brand from structured column or text"""
        try:
            # First priority: use brand column
            if pd.notna(row.get('brand')) and row.get('brand').strip():
                return row['brand'].strip()
            
            # Fallback: extract from text using NER
            if pd.notna(row.get('reviews.text')):
                doc = self.nlp(str(row['reviews.text'])[:500])  # Limit text length
                for ent in doc.ents:
                    if ent.label_ in ['ORG', 'PRODUCT']:
                        return ent.text
            
            return "Unknown"
        except Exception as e:
            logging.error(f"Error in brand extraction: {e}")
            return "Unknown"

    def parse_categories(self, categories_str):
        """Parse categories column into structured format"""
        try:
            if pd.isna(categories_str) or not categories_str:
                return {
                    'main_category': 'Unknown',
                    'sub_category': 'Unknown',
                    'all_categories': []
                }
            
            # Split by comma and clean
            categories = [cat.strip() for cat in str(categories_str).split(',')]
            
            return {
                'main_category': categories[0] if len(categories) > 0 else 'Unknown',
                'sub_category': categories[1] if len(categories) > 1 else 'Unknown',
                'all_categories': categories
            }
        except Exception as e:
            logging.error(f"Error parsing categories: {e}")
            return {
                'main_category': 'Unknown',
                'sub_category': 'Unknown',
                'all_categories': []
            }

    def extract_product_attributes(self, text):
        """Extract product attributes from review text using NER and patterns"""
        try:
            if pd.isna(text):
                return []
            
            attributes = []
            doc = self.nlp(str(text)[:500])
            
            # Extract product-related entities
            for ent in doc.ents:
                if ent.label_ in ['PRODUCT', 'ORG', 'GPE']:
                    attributes.append(ent.text)
            
            # Common product attribute patterns
            patterns = [
                r'\b(flavor|flavour|taste|scent|color|colour|size)\s+:\s+(\w+)',
                r'\b(\w+)\s+(flavor|flavour|taste|scent)',
                r'\b(organic|natural|sugar-free|gluten-free|vegan)\b'
            ]
            
            for pattern in patterns:
                matches = re.findall(pattern, str(text), re.IGNORECASE)
                attributes.extend([match if isinstance(match, str) else ' '.join(match) 
                                 for match in matches])
            
            return list(set(attributes))[:5]  # Return top 5 unique attributes
            
        except Exception as e:
            logging.error(f"Error extracting attributes: {e}")
            return []

    def process_data(self):
        """Main processing function"""
        try:
            logging.info("Starting entity extraction")
            
            # Load data
            df = pd.read_csv(self.data_path)
            logging.info(f"Loaded data with {len(df)} rows")
            
            # Extract brands
            logging.info("Extracting brands...")
            df['extracted_brand'] = df.apply(self.extract_brands, axis=1)
            
            # Parse categories
            logging.info("Parsing categories...")
            category_data = df['categories'].apply(self.parse_categories)
            df['main_category'] = category_data.apply(lambda x: x['main_category'])
            df['sub_category'] = category_data.apply(lambda x: x['sub_category'])
            df['all_categories'] = category_data.apply(lambda x: ','.join(x['all_categories']))
            
            # Extract product attributes from text
            logging.info("Extracting product attributes...")
            df['product_attributes'] = df['reviews.text'].apply(
                lambda x: ','.join(self.extract_product_attributes(x))
            )
            
            # Create a clean dataset with relevant columns
            output_df = df[[
                'id', 'name', 'extracted_brand', 'main_category', 
                'sub_category', 'all_categories', 'product_attributes',
                'reviews.text', 'reviews.rating', 'reviews.title'
            ]].copy()
            
            logging.info("Entity extraction completed")
            return output_df
            
        except Exception as e:
            logging.error("Error during entity extraction")
            raise CustomException(e, sys)

    def save_data(self, df: pd.DataFrame):
        """Save extracted entities"""
        try:
            os.makedirs(self.output_dir, exist_ok=True)
            df.to_csv(self.output_file, index=False)
            logging.info(f"Entity data saved to {self.output_file}")
            
            # Save summary statistics
            summary_file = os.path.join(self.output_dir, "entity_summary.txt")
            with open(summary_file, 'w') as f:
                f.write("Entity Extraction Summary\n")
                f.write("=" * 50 + "\n\n")
                f.write(f"Total Records: {len(df)}\n")
                f.write(f"Unique Brands: {df['extracted_brand'].nunique()}\n")
                f.write(f"Unique Main Categories: {df['main_category'].nunique()}\n\n")
                f.write("Top 10 Brands:\n")
                f.write(df['extracted_brand'].value_counts().head(10).to_string())
                f.write("\n\nTop 10 Categories:\n")
                f.write(df['main_category'].value_counts().head(10).to_string())
            
            logging.info(f"Summary saved to {summary_file}")
            
        except Exception as e:
            logging.error("Error saving entity data")
            raise CustomException(e, sys)


if __name__ == "__main__":
    try:
        extractor = EntityExtraction()
        extracted_df = extractor.process_data()
        extractor.save_data(extracted_df)
        print("Entity extraction completed successfully!")
        
    except Exception as e:
        raise CustomException(e, sys)

Entity extraction completed successfully!
